In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences  # type: ignore
from sklearn.preprocessing import LabelEncoder  # type: ignore
from sklearn.model_selection import train_test_split  # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer  # type: ignore
from keras.models import Sequential   # type: ignore
from keras.layers import Embedding, Flatten, Dense   # type: ignore

In [2]:
data = pd.read_csv("test.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0          im updating my blog because i feel shitty  sadness
1  i never make her separate from me because i do...  sadness
2  i left with my bouquet of red and yellow tulip...      joy
3    i was feeling a little vain when i did this one  sadness
4  i cant walk into a shop anywhere where i do no...     fear


In [3]:

texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
padded_sequences

array([[   0,    0,    0, ...,    1,    2,  488],
       [   0,    0,    0, ...,  391,   21,   72],
       [   0,    0,    0, ...,   36,    1, 1418],
       ...,
       [   0,    0,    0, ...,    7,    9, 4794],
       [   0,    0,    0, ...,   43,   10, 4795],
       [   0,    0,    0, ...,    4,  333, 1415]])

In [8]:
# Encoding the string labels  to iintegers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [9]:
labels

array([4, 4, 2, ..., 2, 2, 1], dtype=int64)

In [10]:
# One hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels) 

In [11]:
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [12]:
# Splitting the data into training and tesing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) +1, output_dim=128, input_length=max_length))

model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

In [27]:
input_text = "i feel so embarrassed"

In [ ]:
# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])

In [29]:
print(predicted_label[0])

sadness
